In [1]:
from numpy import uint16, uint32

In [2]:
with open('save-raw.sav', 'rb') as f:
    dat = f.read()

In [3]:
x = bytearray(dat)

In [4]:
box = x[0x9000:0xa000]
box[0x0FF8:]

bytearray(b'% \x01\x08\x04\x00\x00\x00')

In [5]:
int.from_bytes(box[0x0FF6:0x0FF8], "little")

56886

In [6]:
def calc_hash(sec, sz):
    s = uint32(0)
    for i in range(0, sz // 4):
        s = s + uint32(int.from_bytes(sec[4*i : 4*(i+1)], "little"))
    return uint16(s & 0xFFFF) + uint16(s >> 16)

In [7]:
calc_hash(box, 3968)

<ipython-input-6-15b25b22d2e0>:4: RuntimeWarning: overflow encountered in uint_scalars
  s = s + uint32(int.from_bytes(sec[4*i : 4*(i+1)], "little"))


56886

In [8]:
flag = [0xd6, 0xe9, 0xd7, 0xdf, 0xd9, 0xed, 0xd9, # buckeye
            0x5c, # (
            0xaa, 0xa1, 0xa8, 0xa8, 0xa5, # 9077a
            0xae, # -
            0xa4, 0xe1, 0xe9, 0xa2, 0xa5, 0xa8, 0xa4, # 3mu1473
            0xae, # -
            0xa4, 0xe1, # 3m
            0xae, # -
            0xa5, 0xa2, 0xa2, # 411
            0x5d # )
       ]

In [9]:
for j in range(0, len(flag)):
    box[4 + j*80 : 84 + j*80] = box[4:84] # copy pidgey
    box[12 + j*80 : 22 + j*80] = [flag[j], 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF, 0xFF]

In [10]:
# Uncomment this line to write the new checksum, making the save bootable.
# This would make the challenge too easy, I think
#box[0x0FF6:0x0FF8] = int(calc_hash(box, 3968)).to_bytes(2, "little")
int.from_bytes(box[0x0FF6:0x0FF8], "little")

56886

In [11]:
x[0x9000:0xa000] = box
y = bytes(x)

In [12]:
with open("pocket-monsters.sav", "wb") as fo:
    fo.write(y)